# Part I
## Handling and visualizing data

In this notebook we will download the luminosity data and we will discuss how to visualize them, how to compute secondary features from the modulation of the light sources. 

### Importing libraries
In this notebook we will be using:
 * `numpy` providing basic capabilities of numerical computation in Python
 * `pandas` adding to numpy a database-like access to data stored in a DataFrame data object
 * `matplotlib.pyplot` providing basic plotting capabilities

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

### Downloading the dataset

In [ ]:
dataset_filename = "**YOUR PRESIGNED URL HERE**"
df = pd.read_csv(dataset_filename)

### Exploring the dataset

Let's start by looking at the dataset to grasp some general information.

Visualizing the dataset in tabular form we observe that it has a first column representing the label:
 * 1: *the star is known to not have exoplanets*
 * 2: *the star is known to have exoplanets*
followed by 3197 columns representing the luminosity as a function of time (in intervals of 30 minutes).

In [ ]:
df

The dataset is rather unbalanced as we can immediately visualize with a pie chart.

In [ ]:
df.LABEL.value_counts().plot.pie()
print (df.LABEL.value_counts())

Building the histogram of the luminosities we clearly see that the luminosities spans over a very large range of values covering multiple orders of magnitude. 

In [ ]:
raw = df[df.columns[1:]].values
plt.hist(raw.flatten(), bins=100)
plt.yscale('log')
plt.xlabel("Star captured luminosity [a.u.]")
plt.ylabel("Number of photographies")
plt.show()

### Preprocessing
Since we are mostly interested in effects in the frequency domain, we can clean global luminosity effects interesting the whole evolution of a star. Here we are using our intuition that suggests that the fact a star is more bright should not correlate with the probability of having exoplanets (most likely it is simply closer to Earth...).

Then we remove the average luminosity of each row in our table, and divide by its standard deviation.
In data science jargon, this is named *standardization*.

In [ ]:
data = (raw - raw.mean(axis=1, keepdims=True))/raw.std(axis=1, keepdims=True)

Drawing some preprocessed waveform, it is evident that some of the waveforms have luminosity structures at low frequency that can hardly correlate to exoplanets (though they may indicate binary stars or similar structures...).
For example, look at star #123.

In [ ]:
xAxis = np.arange(data.shape[1]) * 0.5 / 24
plt.figure(figsize=(10,3))
plt.plot(xAxis, data[123], label="Standardized lumi")
plt.grid()
plt.xlabel("Time [days]")
plt.ylabel("Raw luminosity [a.u.]")
plt.legend(title="Star 123")
plt.show()

Using [scipy](https://docs.scipy.org), we can easily define a digital filter to retrieve the low-frequency modulation and possibly subtract it from the signal.
Digital filtering in scipy is implemented through the [filtfilt](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html) function, combined with a function defining the filter transform. For example, *butter* defines a [Butterworth filter](https://en.wikipedia.org/wiki/Butterworth_filter) mimicking an ideal passive electronic filter.

The first argument of the `butter` function is the order of the filter, the second one is the cutoff frequency.
Not that the frequency is expressed as a function of the Nyquist frequency to be independent of the actual sampling rate. 

In this problem the Nyquist frequency is 
$$
f_N = \frac{1}{2}\ \frac{1}{30 \cdot 60} = 2.8\times 10^{-4}\ \mathrm{Hz} = 1\, \mathrm{hour}^{-1}
$$

We design a filter with a cutoff frequency of $0.5\ \mathrm{day}^{-1}$ which roughly correspond to average everything happening within the same day (on Earth).

> **Feel free to play with the filter definition to see how the filter function changes modifying order and cutoff!**

In [ ]:
from scipy.signal import butter, filtfilt

b, a = butter(4, 0.5/24)
filtered = filtfilt(b, a, data)

To visualize the effect of the filter we superpose the original and filtered version of the luminosity stream of star #123.

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(xAxis, data[123], label="Standardized lumi")
plt.plot(xAxis, filtered[123], label="Filterered lumi", linewidth=2)
plt.grid()
plt.xlabel("Time [days]")
plt.ylabel("Raw luminosity [a.u.]")
plt.legend(title="Star 123")
plt.show()

Subtracting the filtered component from the original waveform we obtain a noisy waveform that is easier to analyse programmatically.

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(xAxis, data[123]-filtered[123], label="Fast frequency component")
plt.grid()
plt.xlabel("Time [days]")
plt.ylabel("Raw luminosity [a.u.]")
plt.legend(title="Star 123")
plt.show()

### Correcting experimental biases

The high frequency noise observed in the previous block may be due to experimenal differences in the sensitivity of the camera or on the exposure, rather than on real modifications of the star luminosity. If this is the case, we should see a correlation of the luminosity of most stars at the same acquisition time. 

To qualitatively test this hypotesis, we create figures in which the gray level corresponds to the luminosity of each star at a given instant. The human eye is rather good at noticing patterns and will tell us immediately whether this kind of correlation should be corrected for.

In [ ]:
plt.figure(figsize=(15,8), dpi=100)
plt.imshow(data, aspect='auto', cmap='gray')
plt.xlabel("Time")
plt.ylabel("Star index")
plt.show()

Unfortunately the image appears completely uniform. We *know* that, even after standardization and filtering, our dataset is not constant everywhere. So what is happening? Let's draw a histogram of the standardized and filtered data. 

In [ ]:
plt.hist((data - filtered).flatten(), bins=100)
plt.xlabel("Standardized luminosity")
plt.ylabel("Number of photographies")
plt.yscale('log')
plt.show()

Clearly, there are few pixels with a very high or very low value. To convert the pixel value into a gray scale, pyplot takes the minimum and the maximum of the values to be converted and maps them into black and white, respectively. In this case, however, the vast majority of the pixels is varying on a scale which is tremendously smaller than the difference between minimum and maximum. This makes the figure unreadable.

A possible solution is to discard the outlayers in order to resolve effects within the distribution core.

In practice, we clip the values of the normalized and filtered data
$$
f(x) = \left\{
\begin{array}{ll}
\ell & \mbox{where}\ x < \ell\\
h & \mbox{where}\ x > h\\
x & \mbox{elsewhere}
\end{array}
\right.
$$
where $\ell$ and $h$ are determined inverting the relation
$$
\int_{-\infty}^{\ell} p(L)\mathrm dL = \varepsilon \qquad \int_{h}^{+\infty} p(L)\mathrm dL = \varepsilon,
$$
where $p(L)$ represents the distribution of luminosity.

In practice, in Python it is very simple. The following function uses the functions `np.quantile` and `np.clip` to apply this transform, and then it creates the figure, taking $\varepsilon$ as an argument.


In [ ]:
def build_figure(data, epsilon=None):
    plt.figure(figsize=(15,8), dpi=100)
    if epsilon is not None:
        low, high = np.quantile(data, [epsilon, 1-epsilon])
        data = np.clip(data, low, high)
    plt.imshow(data, aspect='auto', cmap='gray')
    plt.xlabel("Time [$\\frac{1}{2}$ hour]")
    plt.ylabel("Star index")

    plt.show()

build_figure(data - filtered, 1e-2)

### Exercise 1.1 - Final preprocessing ⏳
Use the functions discussed above to replace the definition of `prep_data` below, with a version preprocessed standardizing per time instant and then per star.


## Visualization of the preprocessed waveforms to be classified

We combine in a unique figure the luminosity stream of several stars.
We highlight in *red* the waveforms corresponding to stars with exoplanets.

In [ ]:
plt.figure(figsize=(16,32))

for star_id in range(64):
    plt.plot(xAxis, prep_data[star_id] - 20*star_id, color='#c00' if df.LABEL.values[star_id]==2 else 'black', alpha=1)
    plt.text(0, -20*star_id + 4, str(star_id))
plt.xlabel("Time [days]")
plt.yticks([])
plt.show()

# Exercises

### Exercise 1.2 - Slicing and Visualization ⏳
Consider stars #1 and #7 which have clear effects of exoplanets and try to zoom over the negative luminosity peaks.

### Exercise 1.3 - Bandpass filter ⏳⏳
Having a look to the documentation of `scipy.signal.butter` to define a bandpass filter that enhances the importance of the shadowing peaks. You can use the peaks highlighted in the previous exercise to visualize the effect of the bandpass filter.

Take the minimum of the filtered waveform as a discriminant variable between stars with and without exoplanets.

### Exercise 1.4 - Count threshold crossings ⏳⏳
Define a threshold at, for example, -2, and count the number of times the waveform crosses the threshold.

Crossing the threshold means sample $i$ is above threshold and sample $i+1$ is below.

> **Hint.** You can use `np.count_nonzero` applied to a boolean array to count the occurences of something.

### Exercise 1.4 - Count threshold crossings, with Time Below Threshold ⏳⏳⏳

Extend the previous exercise by requiring that the waveform remains low by a given number of samples. 

> **Hint.** Consider using `np.all` for requiring that all samples in a sequence satisfy the requirement of being below the threshold. 


### Exercise 1.5 - Exploit periodicity with your own C algorithm ⏳⏳⏳⏳

Take a look at [this notebook](../boilerplates/CfromPython.ipynb).
It provides an example of wrapping C code to a Python function.

Starting from that example, try to implement the following algorithm.

Given the brightness $b$ at time $t \in \{t_0, t_1, ..., t_n\}$, $b(t)$ we define an estimator of the period as
$$
\hat T = \mathop{\rm argmin}_{i\, >\, T_0}\left[ \mathop{\rm min}_{j < i} \left( \sum_{k \in \{j, j+i, j + 2i...\}} \frac{b(t_k)}{\left[\mathrm{floor}\left(\frac{n}{i}\right)\right]^{\alpha}} \right)\right]
$$

$T_0$ is the minimal reasonable period (works decently with $T_0 \sim 50 \,\mathrm{hours}$, or $T_0 = 100$ if expressed in 30-minute time steps.

$\alpha$ is a constant that is used to enhance the importance of low frequency periodicity. Should be tuned for best discrimination power in the range $\alpha \in [0,  1]$. 

### Exercise 1.6 - Parallelize the execution on multiple threads ⏳⏳

Take a look to `ThreadPool`(https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool) to parallelize on multiple thread the execution your C algorithm.

> **Note.** You might be tempted to try using `multiprocessing.Pool` instead, DO NOT TRY! 
>
> Forking this application with dependencies on the loaded C library results into indefinite behaviour (ahh... good old C!). 
>
> Optimistically, it would silently crash the Python kernel behind the Jupyter notebook server.

### Exercise 1.7 - Return also the minimum value (not only its index)
So far we have returned a single value: the period estimator obtained with the minimization described above.
In this exercise you are invited to modify the code to return two values.

> **Hint.** A customary technique to return multiple values from a C function is to pass buffer arrays as arguments and filling them from inside the function. Since the arrays are always passed per-reference, the array allocated by the calling function will also be updated. You can use the same approach, defining numpy arrays as buffer in Python code and let them to be filled by the C function with whatever values you wish to retrieve.